Cellule 1 : Imports et connexion au DWH

In [1]:
import os
from dotenv import load_dotenv
import pyodbc
from datetime import datetime, timedelta
import pandas as pd

# Charger les variables d'environnement
load_dotenv()

# Configuration connexion DWH
server = os.getenv('DB_SERVER_DWH')
database = os.getenv('DB_DATABASE_DWH')
username = os.getenv('DB_USERNAME_DWH')
password = os.getenv('DB_PASSWORD_DWH')

# Driver ODBC
driver = '{ODBC Driver 17 for SQL Server}'

# Connexion à la base DWH_E5_projet_AUTO
cnxn = pyodbc.connect(
    'DRIVER=' + driver + 
    ';SERVER=' + server + 
    ';PORT=1433;DATABASE=' + database + 
    ';UID=' + username + 
    ';PWD=' + password
)
cursor = cnxn.cursor()

print("✅ Connexion réussie à la base de données DWH_E5_projet_AUTO")
print(f"📦 Serveur : {server}")
print(f"🗄️  Base : {database}")

✅ Connexion réussie à la base de données DWH_E5_projet_AUTO
📦 Serveur : carter-cash-serveur.database.windows.net
🗄️  Base : DWH_E5_projet_AUTO


Cellule 2 : Vérification si DIM_TEMPS est déjà alimentée


In [2]:
# Vérifier si la table DIM_TEMPS contient déjà des données
cursor.execute("SELECT COUNT(*) FROM DIM_TEMPS")
count = cursor.fetchone()[0]

if count > 0:
    print(f"⚠️  ATTENTION : La table DIM_TEMPS contient déjà {count} enregistrements !")
    print("   Si vous continuez, les doublons seront ignorés grâce à la contrainte UNIQUE sur Date.")
    
    choix = input("\n   Voulez-vous continuer ? (O/N) : ")
    if choix.upper() != 'O':
        print("❌ Opération annulée.")
        cursor.close()
        cnxn.close()
        exit()
else:
    print("✅ La table DIM_TEMPS est vide. Prêt pour l'alimentation.")

✅ La table DIM_TEMPS est vide. Prêt pour l'alimentation.


Cellule 3 : Fonction pour déterminer la saison

In [ ]:
def get_saison(mois):
    """
    Retourne la saison en fonction du mois
    Hiver : décembre, janvier, février
    Printemps : mars, avril, mai
    Été : juin, juillet, août
    Automne : septembre, octobre, novembre
    """
    if mois in [12, 1, 2]:
        return 'Hiver'
    elif mois in [3, 4, 5]:
        return 'Printemps'
    elif mois in [6, 7, 8]:
        return 'Été'
    else:
        return 'Automne'

def get_jour_semaine_fr(jour_index):
    """
    Retourne le nom du jour en français
    0 = Lundi, 6 = Dimanche
    """
    jours = ['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi', 'Samedi', 'Dimanche']
    return jours[jour_index]

print("✅ Fonctions utilitaires créées")

✅ Fonctions utilitaires créées


Cellule 4 : Génération des données pour DIM_TEMPS

In [4]:
print("\n📅 GÉNÉRATION DES DONNÉES POUR DIM_TEMPS")
print("=" * 70)

# Définir la plage de dates
date_debut = datetime(2020, 1, 1)
date_fin = datetime(2030, 12, 31)

# Calculer le nombre de jours
nb_jours = (date_fin - date_debut).days + 1

print(f"📆 Période : {date_debut.strftime('%d/%m/%Y')} au {date_fin.strftime('%d/%m/%Y')}")
print(f"📊 Nombre de jours à générer : {nb_jours}")

# Créer une liste pour stocker toutes les lignes
donnees_temps = []

# Générer les données jour par jour
date_courante = date_debut
compteur = 0

print("\n⏳ Génération en cours...")

while date_courante <= date_fin:
    # Extraire les composants de la date
    jour = date_courante.day
    mois = date_courante.month
    annee = date_courante.year
    trimestre = (mois - 1) // 3 + 1
    jour_semaine_index = date_courante.weekday()  # 0 = Lundi, 6 = Dimanche
    jour_semaine = get_jour_semaine_fr(jour_semaine_index)
    est_weekend = 1 if jour_semaine_index >= 5 else 0  # Samedi=5, Dimanche=6
    saison = get_saison(mois)
    
    # Ajouter à la liste
    donnees_temps.append({
        'Date': date_courante.strftime('%Y-%m-%d'),
        'Jour': jour,
        'Mois': mois,
        'Trimestre': trimestre,
        'Annee': annee,
        'Jour_Semaine': jour_semaine,
        'Est_Weekend': est_weekend,
        'Saison': saison
    })
    
    # Passer au jour suivant
    date_courante += timedelta(days=1)
    compteur += 1
    
    # Afficher la progression tous les 500 jours
    if compteur % 500 == 0:
        print(f"   ⏳ {compteur} jours générés...")

print(f"✅ {len(donnees_temps)} jours générés avec succès !")

# Créer un DataFrame pandas pour visualiser un échantillon
df_temps = pd.DataFrame(donnees_temps)
print("\n📋 Aperçu des 10 premiers enregistrements :")
print(df_temps.head(10))


📅 GÉNÉRATION DES DONNÉES POUR DIM_TEMPS
📆 Période : 01/01/2020 au 31/12/2030
📊 Nombre de jours à générer : 4018

⏳ Génération en cours...
   ⏳ 500 jours générés...
   ⏳ 1000 jours générés...
   ⏳ 1500 jours générés...
   ⏳ 2000 jours générés...
   ⏳ 2500 jours générés...
   ⏳ 3000 jours générés...
   ⏳ 3500 jours générés...
   ⏳ 4000 jours générés...
✅ 4018 jours générés avec succès !

📋 Aperçu des 10 premiers enregistrements :
         Date  Jour  Mois  Trimestre  Annee Jour_Semaine  Est_Weekend Saison
0  2020-01-01     1     1          1   2020     Mercredi            0  Hiver
1  2020-01-02     2     1          1   2020        Jeudi            0  Hiver
2  2020-01-03     3     1          1   2020     Vendredi            0  Hiver
3  2020-01-04     4     1          1   2020       Samedi            1  Hiver
4  2020-01-05     5     1          1   2020     Dimanche            1  Hiver
5  2020-01-06     6     1          1   2020        Lundi            0  Hiver
6  2020-01-07     7     1   

Cellule 5 : Insertion des données dans DIM_TEMPS


In [5]:
print("\n💾 INSERTION DES DONNÉES DANS DIM_TEMPS")
print("=" * 70)

# Requête d'insertion
insert_query = """
INSERT INTO DIM_TEMPS (Date, Jour, Mois, Trimestre, Annee, Jour_Semaine, Est_Weekend, Saison)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
"""

# Compteurs
inseres = 0
ignores = 0
erreurs = 0

print("⏳ Insertion en cours...")

for ligne in donnees_temps:
    try:
        cursor.execute(
            insert_query,
            ligne['Date'],
            ligne['Jour'],
            ligne['Mois'],
            ligne['Trimestre'],
            ligne['Annee'],
            ligne['Jour_Semaine'],
            ligne['Est_Weekend'],
            ligne['Saison']
        )
        inseres += 1
        
        # Afficher la progression tous les 500 insertions
        if inseres % 500 == 0:
            print(f"   ✓ {inseres} lignes insérées...")
            cnxn.commit()  # Commit intermédiaire pour éviter les timeouts
            
    except pyodbc.IntegrityError:
        # Date en doublon (contrainte UNIQUE)
        ignores += 1
    except Exception as e:
        erreurs += 1
        print(f"   ⚠️  Erreur lors de l'insertion de {ligne['Date']} : {e}")

# Commit final
cnxn.commit()

print("\n" + "=" * 70)
print("✅ INSERTION TERMINÉE")
print("=" * 70)
print(f"   ✓ Lignes insérées : {inseres}")
print(f"   ⚠️  Lignes ignorées (doublons) : {ignores}")
print(f"   ❌ Erreurs : {erreurs}")
print("=" * 70)


💾 INSERTION DES DONNÉES DANS DIM_TEMPS
⏳ Insertion en cours...
   ✓ 500 lignes insérées...
   ✓ 1000 lignes insérées...
   ✓ 1500 lignes insérées...
   ✓ 2000 lignes insérées...
   ✓ 2500 lignes insérées...
   ✓ 3000 lignes insérées...
   ✓ 3500 lignes insérées...
   ✓ 4000 lignes insérées...

✅ INSERTION TERMINÉE
   ✓ Lignes insérées : 4018
   ⚠️  Lignes ignorées (doublons) : 0
   ❌ Erreurs : 0


Cellule 6 : Vérification et statistiques


In [6]:
print("\n🔍 VÉRIFICATION ET STATISTIQUES")
print("=" * 70)

# Compter le nombre total d'enregistrements
cursor.execute("SELECT COUNT(*) FROM DIM_TEMPS")
total = cursor.fetchone()[0]
print(f"📊 Nombre total d'enregistrements dans DIM_TEMPS : {total}")

# Statistiques par année
cursor.execute("""
SELECT Annee, COUNT(*) as Nb_Jours
FROM DIM_TEMPS
GROUP BY Annee
ORDER BY Annee
""")

print("\n📅 Répartition par année :")
for row in cursor.fetchall():
    print(f"   • {row[0]} : {row[1]} jours")

# Statistiques par saison
cursor.execute("""
SELECT Saison, COUNT(*) as Nb_Jours
FROM DIM_TEMPS
GROUP BY Saison
ORDER BY CASE Saison 
    WHEN 'Printemps' THEN 1 
    WHEN 'Été' THEN 2 
    WHEN 'Automne' THEN 3 
    WHEN 'Hiver' THEN 4 
END
""")

print("\n🌸 Répartition par saison :")
for row in cursor.fetchall():
    print(f"   • {row[0]} : {row[1]} jours")

# Nombre de weekends
cursor.execute("SELECT COUNT(*) FROM DIM_TEMPS WHERE Est_Weekend = 1")
nb_weekends = cursor.fetchone()[0]
print(f"\n🎉 Nombre de jours de weekend : {nb_weekends}")

# Afficher quelques exemples
cursor.execute("""
SELECT TOP 5 Date, Jour_Semaine, Saison, Est_Weekend
FROM DIM_TEMPS
ORDER BY Date DESC
""")

print("\n📋 Exemples d'enregistrements récents :")
for row in cursor.fetchall():
    weekend_label = "🎉 Weekend" if row[3] == 1 else "Semaine"
    print(f"   • {row[0]} - {row[1]} - {row[2]} - {weekend_label}")

print("=" * 70)


🔍 VÉRIFICATION ET STATISTIQUES
📊 Nombre total d'enregistrements dans DIM_TEMPS : 4018

📅 Répartition par année :
   • 2020 : 366 jours
   • 2021 : 365 jours
   • 2022 : 365 jours
   • 2023 : 365 jours
   • 2024 : 366 jours
   • 2025 : 365 jours
   • 2026 : 365 jours
   • 2027 : 365 jours
   • 2028 : 366 jours
   • 2029 : 365 jours
   • 2030 : 365 jours

🌸 Répartition par saison :
   • Printemps : 1012 jours
   • Été : 1012 jours
   • Automne : 1001 jours
   • Hiver : 993 jours

🎉 Nombre de jours de weekend : 1148

📋 Exemples d'enregistrements récents :
   • 2030-12-31 - Mardi - Hiver - Semaine
   • 2030-12-30 - Lundi - Hiver - Semaine
   • 2030-12-29 - Dimanche - Hiver - 🎉 Weekend
   • 2030-12-28 - Samedi - Hiver - 🎉 Weekend
   • 2030-12-27 - Vendredi - Hiver - Semaine


Cellule 7 : Fermeture de la connexion


In [7]:
# Fermeture de la connexion
cursor.close()
cnxn.close()

print("\n" + "=" * 70)
print("✅ CONNEXION FERMÉE")
print("🎉 ALIMENTATION DE DIM_TEMPS TERMINÉE AVEC SUCCÈS !")
print("=" * 70)
print("\n💡 La table DIM_TEMPS est maintenant prête pour les processus ETL.")
print("   Vous pouvez maintenant charger les tables de faits.")
print("=" * 70)


✅ CONNEXION FERMÉE
🎉 ALIMENTATION DE DIM_TEMPS TERMINÉE AVEC SUCCÈS !

💡 La table DIM_TEMPS est maintenant prête pour les processus ETL.
   Vous pouvez maintenant charger les tables de faits.
